# Countermeasure tests

In [129]:
from pyecsca.sca.target.ectester import KeypairEnum, ParameterEnum, CurveEnum, KeyEnum, KeyClassEnum, KeyBuildEnum, KeyAgreementEnum, SignatureEnum, TransformationEnum
from pyecsca.ec.params import load_params_ectester
from pyecsca.sca.target.ectester import ECTesterTargetPCSC, KeyAgreementEnum
from pyecsca.sca.target.PCSC import PCSCTarget
from smartcard.System import readers
import numpy as np
from time import sleep
from utils import *
import os

In [132]:
rs = readers()
for reader in rs:
    if "Gemalto" in reader.name:
        break
target = ECTesterTargetPCSC(reader)

In [133]:
target.connect()

In [134]:
target.select_applet()

True

In [135]:
CARD = "I1"

In [136]:
atr = target.atr.hex()
card_map = {
    "3bd518ff8191fe1fc38073c8211309": "A1",
    "3bb89600c00831fe45ffff1154305023006a": "I1",
    "3bfe1800008031fe45803180664090a5102e1083019000f2": "I2",
    "3bf81800ff8131fe454a434f507632343143": "N1",
    "3bf81300008131fe454a434f5076323431b7": "N2N9",
    "3b9495810146545601c4": "N4N10",
    "3bd518ff8191fe1fc38073c821100a": "N6",
    "3b9c9580811f039067464a01005404f272fe00c0": "F1",
    "3b90958011fe6a": "F2",
    "3b9f95803fc7a08031e073fa21106300000083f09000bb": "S1S2",
    "3bf91300008131fe454a434f503234325233a2":"N8",
    "3bf99600008131fe4553434537200e00202028":"G1"
}
print(f"{card_map[atr]} connected, {CARD} selected")
assert CARD in card_map[atr]


I1 connected, I1 selected


## Test cofactor

In [314]:
params = load_params_ectester("tests/testcofactor/cofactor256p4_h.csv", "affine")
params_csv = read_curve_params("tests/testcofactor/cofactor256p4_h.csv")
point = get_point_bytes("tests/testcofactor/point_4n.csv")
with open("tests/testcofactor/key.csv") as f:
    key = int(f.read(),16)

In [358]:
params = load_params_ectester("tests/testcofactor/weakcurve.csv", "affine")
params_csv = read_curve_params("tests/testcofactor/weakcurve.csv")
point = get_point_bytes("tests/testcofactor/weakcurve_point_4n.csv")
with open("tests/testcofactor/key.csv") as f:
    key = int(f.read(),16)

In [359]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

AllocateResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [360]:
target.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH_PLAIN)

AllocateKaResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [361]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [362]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [363]:
result = target.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                         True,
                         TransformationEnum.NONE,
                         KeyAgreementEnum.ALG_EC_SVDP_DH_PLAIN,
                         point) # pubkey as bytes
if not result.success:
    print(result)
result_lines = [serialize_ecdh_response(result,params_csv,point,key)]

In [364]:
result.resp

ResponseAPDU(data=b"\x90\x00\x00 S\x9d\x8b\xeb\xdfI\x892\xf1\x90\r\xbb\xca\x10\xb7\xdfC\x98\xf7\xb5\x08\x86\xd9\x87\x0fa'\x93\xa4\x0b>f", sw=36864)

In [365]:
header = "success;error;secret[SHA1];priv;pub;curve;params;apdu;sws"
filename = f"results/{CARD}/testcofactor/ecdh_plain2.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    f.write(f"{header}\n")
    for line in result_lines:
        f.write(f"{line}\n")

### Test3n - ECDH

In [6]:
params = load_params_ectester("tests/test3n/curve_prime_gen.csv", "affine")
params_csv = read_curve_params("tests/test3n/curve_prime_gen.csv")
point = get_point_bytes("tests/test3n/point_3n.csv")
with open("tests/test3n/key.csv") as f:
    key = int(f.read(),16)

In [8]:
samples = 100

In [ ]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [9]:
target.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

AllocateKaResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [10]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [11]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [12]:
result_lines = []

for _ in range(samples):
    result = target.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                         True,
                         TransformationEnum.NONE,
                         KeyAgreementEnum.ALG_EC_SVDP_DH,
                         point) # pubkey as bytes
    if not result.success:
        print(result)
    result_lines.append(serialize_ecdh_response(result,params_csv,point,key))

In [167]:
target.cleanup()

CleanupResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [1]:
target.disconnect()

NameError: name 'target' is not defined

### Test3n - ECDSA

In [7]:
target.connect()

In [90]:
params = load_params_ectester("tests/test3n/curve.csv", "affine")
params_csv = read_curve_params("tests/test3n/curve.csv")
with open("tests/test3n/key.csv") as f:
    key = int(f.read(),16)
point_bytes = get_point_bytes("tests/test3n/point_key.csv")
point = params.curve.decode_point(point_bytes)

In [187]:
samples = 10
fixed_key = False
data = bytes.fromhex("e8c86c9596e46403c211206617b8bcdd160a673b0b1304869f5559e3afe99d79")

In [188]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

AllocateResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [ ]:
target.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

In [189]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [153]:
if fixed_key:
    keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
    keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point))
    print(target.set(KeypairEnum.KEYPAIR_LOCAL,
                 CurveEnum.external,
                 ParameterEnum.KEYPAIR,
                 keypair_dict))
else:
    print(target.generate(KeypairEnum.KEYPAIR_LOCAL))
    export = target.export(KeypairEnum.KEYPAIR_LOCAL, KeyEnum.BOTH,ParameterEnum.KEYPAIR)
    print(export)
    key = int(export.params[1].hex(),16)
    point = params.curve.decode_point(export.params[0])
    point_bytes = export.params[0]

GenerateResponse(sws=[0x2], sw=0x9000, success=False, error=False)
ExportResponse(sws=[0x2], sw=0x9000, success=False, error=True, keypair=KEYPAIR_LOCAL, key=BOTH, params=KEYPAIR)


ValueError: invalid literal for int() with base 16: ''

In [ ]:
result_lines = []
error = False
for _ in range(samples):
    sign_response = target.ecdsa_sign(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    if error or not sign_response.success:
        print(f"sign: {sign_response.sws}",end=", ")
        error = True
    signature = sign_response.signature
    verify_response = target.ecdsa_verify(KeypairEnum.KEYPAIR_LOCAL,SignatureEnum.ALG_ECDSA_SHA,signature,data)
    if error or not verify_response.success:
        print(f"verify: {verify_response.sws}",end=", ")
        error = True
    # verify_response = target.ecdsa(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    try:
        result_lines.append(serialize_ecdsa_response(sign_response,data,params,key,params_csv,point_bytes, verify_response.success))
    except:
        continue

In [ ]:
result_lines

In [115]:
filename = f"results/{CARD}/test3n/ecdsa.csv" if not fixed_key else f"results/{CARD}/test3n/ecdsa_fixed.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [216]:
target.cleanup()


KeyboardInterrupt: 

In [ ]:
target.disconnect()

### Test3n - Keygen

In [213]:
params = load_params_ectester("tests/test3n/curve.csv", "affine")
params_csv = read_curve_params("tests/test3n/curve.csv")

In [214]:
samples = 1000

In [215]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

AllocateResponse(sws=[0x3], sw=0x9000, success=False, error=False)

In [195]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [ ]:
result_lines = []
for _ in range(samples):
    generate_response = target.generate(KeypairEnum.KEYPAIR_LOCAL)
    error = False
    if not generate_response.success:
        print(f"generate: {generate_response.sws}",end=", ")
        error = True
    export = target.export(KeypairEnum.KEYPAIR_LOCAL, KeyEnum.BOTH,ParameterEnum.KEYPAIR)
    if not export.success:
        print(f"export: {export.sws}",end=", ")
        error = True
    
    try:
        key = int(export.params[1].hex(),16)
        point = params.curve.decode_point(export.params[0])
        point_bytes = export.params[0]
        result_lines.append(serialize_keygen_response(generate_response,key,params_csv,point_bytes))
    except:
        continue

In [ ]:
result_lines

In [204]:
header = "success;error;priv;pub;curve;params;apdu;sws"
filename = f"results/{CARD}/test3n/keygen.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    f.write(f"{header}\n")
    for line in result_lines:
        f.write(f"{line}\n")

In [ ]:
target.disconnect()

### Testinverse - ECDH

In [51]:
params = load_params_ectester("tests/testinverse/cofactor256p11_full.csv", "affine")
params_csv = read_curve_params("tests/testinverse/cofactor256p11_full.csv")
point_bytes = get_point_bytes("tests/testinverse/point_11n.csv")
with open("tests/testinverse/key.csv") as f:
    key = int(f.read(),16)

In [52]:
samples = 100

In [53]:
target.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

AllocateKaResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [54]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [55]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [56]:
result_lines = []

for _ in range(samples):
    result = target.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                         True,
                         TransformationEnum.NONE,
                         KeyAgreementEnum.ALG_EC_SVDP_DH,
                         point_bytes) # pubkey as bytes
    if not result.success:
        print(result)
    result_lines.append(serialize_ecdh_response(result,params_csv,point_bytes,key))

In [58]:
filename = f"results/{CARD}/testinverse/ecdh.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [59]:
target.cleanup()

CleanupResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [ ]:
target.disconnect()

### Testinverse - ECDSA

In [22]:
target.connect()

In [7]:
params = load_params_ectester("tests/testinverse/cofactor256p11_full.csv", "affine")
params_csv = read_curve_params("tests/testinverse/cofactor256p11_full.csv")
point_bytes = get_point_bytes("tests/testinverse/point_11n.csv")
point = params.curve.decode_point(point_bytes)

with open("tests/testinverse/key.csv") as f:
    key = int(f.read(),16)

In [8]:
samples = 100
fixed_key = False
data = bytes.fromhex("e8c86c9596e46403c211206617b8bcdd160a673b0b1304869f5559e3afe99d79")

In [9]:
target.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

AllocateSigResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [10]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [11]:
if fixed_key:
    keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
    keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point))
    print(target.set(KeypairEnum.KEYPAIR_LOCAL,
                 CurveEnum.external,
                 ParameterEnum.KEYPAIR,
                 keypair_dict))
else:
    print(target.generate(KeypairEnum.KEYPAIR_LOCAL))
    export = target.export(KeypairEnum.KEYPAIR_LOCAL, KeyEnum.BOTH,ParameterEnum.KEYPAIR)
    print(export)
    key = int(export.params[1].hex(),16)
    point = params.curve.decode_point(export.params[0])
    point_bytes = export.params[0]

GenerateResponse(sws=[0x9000], sw=0x9000, success=True, error=False)
ExportResponse(sws=[0x9000], sw=0x9000, success=True, error=False, keypair=KEYPAIR_LOCAL, key=BOTH, params=KEYPAIR)


In [12]:
result_lines = []

for _ in range(samples):
    sign_response = target.ecdsa(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    if not sign_response.success:
        print(sign_response)
    # verify_response = target.ecdsa(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    result_lines.append(serialize_ecdsa_response(sign_response,data,params,key,params_csv,point_bytes))

ECDSAResponse(sws=[0xee1], sw=0x9000, success=False, error=False, sig=3045022100b7a1ad7daaa91c559046f314f54f556b0755a0de0546023c43e7604fe9ed1fcc022053cfcb6bd1d819176bb27a7f01c3b9b278e15855f3fb578ff5d80a0a8dcedc2c)
ECDSAResponse(sws=[0xee1], sw=0x9000, success=False, error=False, sig=304402207b00aaa06e824519f0fc475641999088b987c5032d93c4faadaf6ed9849d864302205784ebecbee557ab096610c39717ddaffcdf115d3d0c3d824a28ebd0525c0ad6)
ECDSAResponse(sws=[0xee1], sw=0x9000, success=False, error=False, sig=30450220246ea8a99be18caa06ad09f34a06ea024bbacdc6583d9e300f14e59c1b77753a0221008460644a4c21ef6947876afd0fae698a9cb2b89375cc1e7ba1724fa32ab9aaa8)
ECDSAResponse(sws=[0xee1], sw=0x9000, success=False, error=False, sig=3045022100ad9e83310d27ec5f7abca5c692fd5f471d77a3d941b7fcfc83c3b8d4a56d2c3202204423d7d357f24f7f52c82b001bb8597f6dab0286d3d735558c3ae19e9d017501)
ECDSAResponse(sws=[0xee1], sw=0x9000, success=False, error=False, sig=30440220565efd10fdf1f946f23cf3c02117b3eb5c78404ba884e77ad7653ec0422144170220

In [13]:
filename = f"results/{CARD}/testinverse/ecdsa.csv" if not fixed_key else f"results/{CARD}/testinverse/ecdsa_fixedkey.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [14]:
target.cleanup()

CleanupResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [ ]:
target.disconnect()

### Test3n - Keygen

In [15]:
divisor = 11
params = load_params_ectester(f"tests/testinverse/cofactor256p{divisor}_full.csv", "affine")
params_csv = read_curve_params(f"tests/testinverse/cofactor256p{divisor}_full.csv")

In [16]:
samples = 10

In [17]:
target.allocate_ka(KeypairEnum.KEYPAIR_LOCAL)

AllocateKaResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [18]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [19]:
result_lines = []
for _ in range(samples):
    generate_response = target.generate(KeypairEnum.KEYPAIR_LOCAL)
    if not generate_response.success:
        print(generate_response)
    export = target.export(KeypairEnum.KEYPAIR_LOCAL, KeyEnum.BOTH,ParameterEnum.KEYPAIR)
    if not export.success:
        print(export)
    key = int(export.params[1].hex(),16)
    point = params.curve.decode_point(export.params[0])
    point_bytes = export.params[0]
    result_lines.append(serialize_keygen_response(generate_response,key,params_csv,point_bytes))

In [20]:
filename = f"results/{CARD}/testinverse/keygen_{divisor}.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [21]:
target.disconnect()

### Test10 - ECDH

In [30]:
params = load_params_ectester("tests/testk10/secp256r1.csv", "affine")
params_csv = read_curve_params("tests/testk10/secp256r1.csv")
point_bytes = get_point_bytes("tests/testk10/secgpoint.csv")
with open("tests/testk10/key_10.csv") as f:
    key = int(f.read(),16)

In [31]:
samples = 100

In [32]:
target.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

AllocateKaResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [33]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [34]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [35]:
result_lines = []

for _ in range(samples):
    result = target.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                         True,
                         TransformationEnum.NONE,
                         KeyAgreementEnum.ALG_EC_SVDP_DH,
                         point_bytes) # pubkey as bytes
    if not result.success:
        print(result)
    result_lines.append(serialize_ecdh_response(result,params_csv,point_bytes,key))

CardConnectionException: Failed to transmit with protocol T1. Transaction failed.: Transaction failed. (0x80100016)

In [36]:
filename = f"results/{CARD}/testk10/ecdh.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [38]:
result.params

[b"\xf7\xd1\xee\x97Nz*?'v\x98\x96\x14A\x1e\xc8GD\xe3$"]

In [ ]:
target.cleanup()


In [46]:
target.disconnect()

### Test n+epsilon

In [38]:
params = load_params_ectester("tests/testdn/weakcurve_32_n_x.csv", "affine")
params_csv = read_curve_params("tests/testdn/weakcurve_32_n_x.csv")
point_bytes = get_point_bytes("tests/testdn/key_point.csv")
point = params.curve.decode_point(point_bytes)

with open("tests/testdn/key.csv") as f:
    key = int(f.read(),16)

In [39]:
samples = 100
data = bytes.fromhex("e8c86c9596e46403c211206617b8bcdd160a673b0b1304869f5559e3afe99d79")

In [40]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

AllocateResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [41]:
target.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

AllocateSigResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [42]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [43]:
keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point))
print(target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.KEYPAIR,
             keypair_dict))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)


In [44]:
result_lines = []
error = False
for _ in range(samples):
    sign_response = target.ecdsa_sign(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    if error or not sign_response.success:
        print(f"sign: {sign_response.sws}",end=", ")
        error = True
    signature = sign_response.signature
    try:
        result_lines.append(serialize_ecdsa_response(sign_response,data,params,key,params_csv,point_bytes, None))
    except:
        continue

In [ ]:
result_lines

In [45]:
filename = f"results/{CARD}/testdn/ecdsa_xfixed.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
header = "success;error;signature;valid;data;nonce;priv;pub;curve;params;apdu;sws"
with open(filename,"w") as f:
    f.write(f"{header}\n")
    for line in result_lines:
        f.write(f"{line}\n")

### Test small subgroup attack on verify

In [137]:
from pyecsca.ec.mod import Mod, RawMod

In [138]:
from pyasn1.codec.der.decoder import decode
from pyasn1.type.univ import Sequence

def parse_ecdsa_signature(signature_der):
    decoded_signature, _ = decode(signature_der, asn1Spec=Sequence())
    r = int(decoded_signature[0])
    s = int(decoded_signature[1])
    return r, s

def is_r_even(signature_der, n):
    r,s = parse_ecdsa_signature(signature_der)
    scalar = int(r*RawMod(s,n).inverse())
    return scalar%2==0 and r%2==0

In [139]:
params = load_params_ectester("tests/verify2/cofactor256p2_h.csv", "affine")
params_csv = read_curve_params("tests/verify2/cofactor256p2_h.csv")
point_bytes = get_point_bytes("tests/verify2/key_point.csv")
point = params.curve.decode_point(point_bytes)
point_wrong_bytes = get_point_bytes("tests/verify2/key_point_wrong.csv")
point_wrong = params.curve.decode_point(point_wrong_bytes)
point_realwrong_bytes = get_point_bytes("tests/verify2/key_point_realwrong.csv")
point_realwrong = params.curve.decode_point(point_realwrong_bytes)
with open("tests/testdn/key.csv") as f:
    key = int(f.read(),16)

In [140]:
data = bytes.fromhex("e8c86c9596e46403c211206617b8bcdd160a673b0b1304869f5559e3afe99d79")

In [141]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

AllocateResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [142]:
target.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

AllocateSigResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [143]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)

In [144]:
keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point))
print(target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.KEYPAIR,
             keypair_dict))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)


In [146]:
sign_response = target.ecdsa_sign(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
print(is_r_even(sign_response.signature,params.full_order//2))
sign_response

True


ECDSAResponse(sws=[0x9000], sw=0x9000, success=True, error=False, sig=304402203f74cc77f114f8e8b8909e8fb42d5037907058460c0d4f442a904e918b490f380220097b869aa3a4925a60e51f7cf223ced8337b7ef7974d797dd94f41d4874192dd)

In [147]:
target.ecdsa_verify(KeypairEnum.KEYPAIR_LOCAL,SignatureEnum.ALG_ECDSA_SHA,sign_response.signature,data)

ECDSAResponse(sws=[0x9000], sw=0x9000, success=True, error=False, sig=)

In [148]:
keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point_wrong))
print(target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.KEYPAIR,
             keypair_dict))

SetResponse(sws=[0x9000], sw=0x9000, success=True, error=False)


In [149]:
target.ecdsa_verify(KeypairEnum.KEYPAIR_LOCAL,SignatureEnum.ALG_ECDSA_SHA,sign_response.signature,data)

ECDSAResponse(sws=[0xee1], sw=0x9000, success=False, error=False, sig=)